In [1]:
%load_ext autoreload
%autoreload

import os
import sys
import random
import django
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from datetime import datetime
import time
from tqdm import tqdm_notebook
from itertools import combinations

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.apps import apps
from django_currentuser.middleware import get_current_user

In [2]:
comb = combinations(m.Pix.objects.all(), 2); comb

In [3]:
%%time
random.sample(list(combinations(m.Pix.objects.values_list('id', flat=True), 2)), 500)

CPU times: user 576 ms, sys: 147 ms, total: 723 ms
Wall time: 904 ms


[(1706, 1947),
 (2160, 3435),
 (1892, 2927),
 (891, 3784),
 (2040, 2252),
 (2219, 3460),
 (3991, 4005),
 (4109, 4387),
 (1300, 3930),
 (982, 1933),
 (851, 4272),
 (2095, 3278),
 (2067, 2659),
 (203, 3061),
 (174, 3780),
 (3190, 3612),
 (196, 2435),
 (625, 2462),
 (1939, 3191),
 (1225, 2011),
 (1802, 2594),
 (337, 4280),
 (2571, 4342),
 (1253, 3930),
 (2839, 4133),
 (1552, 4012),
 (721, 1094),
 (878, 4154),
 (1740, 2965),
 (570, 3982),
 (611, 2307),
 (3125, 3130),
 (555, 2384),
 (326, 2179),
 (804, 3959),
 (1387, 2550),
 (3082, 4098),
 (1007, 1089),
 (246, 4312),
 (2008, 2453),
 (1426, 1509),
 (3821, 3869),
 (236, 1958),
 (551, 3019),
 (557, 1642),
 (2220, 2541),
 (1973, 2054),
 (1693, 2440),
 (788, 702),
 (1590, 2064),
 (1434, 4410),
 (1716, 1912),
 (474, 3918),
 (1387, 1621),
 (1396, 4057),
 (356, 3367),
 (2846, 3339),
 (3957, 4083),
 (2713, 3751),
 (1861, 4068),
 (336, 1725),
 (1523, 3948),
 (1033, 1466),
 (363, 2722),
 (361, 3546),
 (827, 2409),
 (2723, 2787),
 (1406, 3807),
 (824, 

In [4]:
%%time
pixids = list(m.Pix.objects.values_list('id', flat=True))
my_sample = set()
needed_size = 500

while len(my_sample) < needed_size:
    my_sample.add(frozenset(random.sample(pixids, 2)))
    
my_sample

CPU times: user 4.86 ms, sys: 1.02 ms, total: 5.88 ms
Wall time: 187 ms


{frozenset({534, 2110}),
 frozenset({520, 2641}),
 frozenset({1769, 3875}),
 frozenset({2708, 3438}),
 frozenset({659, 3987}),
 frozenset({1787, 2140}),
 frozenset({452, 2775}),
 frozenset({954, 4233}),
 frozenset({1111, 3528}),
 frozenset({1780, 2478}),
 frozenset({751, 1891}),
 frozenset({1548, 1742}),
 frozenset({861, 2893}),
 frozenset({1387, 4005}),
 frozenset({618, 3111}),
 frozenset({1621, 2422}),
 frozenset({2311, 4177}),
 frozenset({2946, 4028}),
 frozenset({483, 3815}),
 frozenset({212, 417}),
 frozenset({2325, 3409}),
 frozenset({1456, 3256}),
 frozenset({890, 4209}),
 frozenset({516, 4098}),
 frozenset({1878, 3498}),
 frozenset({902, 1323}),
 frozenset({1450, 2693}),
 frozenset({3593, 3990}),
 frozenset({900, 4380}),
 frozenset({1558, 2212}),
 frozenset({868, 2018}),
 frozenset({1142, 2171}),
 frozenset({2860, 3679}),
 frozenset({402, 2343}),
 frozenset({272, 2656}),
 frozenset({1126, 3298}),
 frozenset({3311, 3879}),
 frozenset({2587, 4124}),
 frozenset({1953, 2892}),
 fro

In [5]:
[list(e) for e in my_sample]

[[2110, 534],
 [520, 2641],
 [1769, 3875],
 [2708, 3438],
 [3987, 659],
 [1787, 2140],
 [452, 2775],
 [4233, 954],
 [3528, 1111],
 [1780, 2478],
 [1891, 751],
 [1548, 1742],
 [861, 2893],
 [1387, 4005],
 [618, 3111],
 [1621, 2422],
 [4177, 2311],
 [2946, 4028],
 [483, 3815],
 [417, 212],
 [3409, 2325],
 [3256, 1456],
 [4209, 890],
 [4098, 516],
 [3498, 1878],
 [1323, 902],
 [1450, 2693],
 [3593, 3990],
 [900, 4380],
 [2212, 1558],
 [2018, 868],
 [2171, 1142],
 [2860, 3679],
 [402, 2343],
 [272, 2656],
 [3298, 1126],
 [3879, 3311],
 [2587, 4124],
 [1953, 2892],
 [590, 951],
 [465, 1797],
 [3792, 680],
 [3875, 4229],
 [3462, 4223],
 [1675, 3494],
 [1459, 1188],
 [3848, 1565],
 [3164, 287],
 [2328, 4329],
 [4058, 2558],
 [2065, 3554],
 [392, 1711],
 [593, 2955],
 [2618, 4095],
 [2777, 511],
 [152, 569],
 [2827, 2971],
 [448, 3925],
 [2889, 2965],
 [904, 2324],
 [912, 1293],
 [4348, 3926],
 [201, 1285],
 [3012, 1253],
 [257, 615],
 [241, 1011],
 [1260, 3333],
 [401, 1779],
 [1482, 250],
 [